In [1]:
import numpy as np

In [2]:
D = 10
cf_num = 10
search_range = [-100, 100]

In [3]:
shift_data = np.loadtxt(fname = 'shift_data.txt')
M_D10 = np.loadtxt(fname = 'M_D10.txt').reshape((cf_num, D, D))
M1 = M_D10[0].reshape((-1))
o = shift_data.reshape((-1))[:cf_num*D]

In [4]:
# Shift Function
def shiftfunc(x, o):
    
    xshift = x - o[:x.shape[0]]
        
    return xshift

In [5]:
# Rotate Function
def rotatefunc(x, M1):
    
    xrot = np.zeros((x.shape[0]))
    
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            xrot[i] = xrot[i] + x[j] * M1[i*D+j]
    
    return xrot

In [6]:
# Unimodal Function
def sphere(x, o, bias = -1400):
    
    """ Sphere function (CEC2013). """
       
    # Shift
    xshift = shiftfunc(x, o)
    
    # Sphere
    f = sum(xshift**2) + bias
    
    return f

In [7]:
sphere(x = np.array([-10, 20, 30, 100, 5, 6, 7, 8, 9, 10]), o = o)

15639.430369224727

In [8]:
sphere(x = np.zeros((D)), o = o)

17398.270025643684

In [9]:
sphere(x = np.ones((D)), o = o)

17297.32765056778

In [10]:
# Basic Multimodal Function
def rosenbrock(x, o, M1, a = 1, b = 100, bias = -900):
    
    """ Rotated Rosenbrock's function (CEC2013)."""
    
    # Shift
    xshift = shiftfunc(x, o)
    # Shrink to the orginal search range
    xshift = 2.048/100*xshift
    # Rotate
    xrot = rotatefunc(xshift, M1)
    # Shift to origin
    xrot = xrot + 1
    
    # Rosenbrock
    f = sum(b*(xrot[1:]-xrot[:-1]**2.0)**2.0 + (a-xrot[:-1])**2.0) + bias
    
    return f

In [11]:
rosenbrock(x = np.ones((D)), o = o, M1 = M1)

862.8384586846287

In [12]:
rosenbrock(x = np.zeros((D)), o = o, M1 = M1)

961.2132235027593

In [13]:
rosenbrock(x = np.array([-10, 20, 30, 100, 5, 6, 7, 8, 9, 10]), o = o, M1 = M1)

-146.22011188007627